In [15]:
import redis
import os
import sys
sys.path.append(os.path.abspath('..'))
from src.utils.redis_client import RedisClient
from src.utils.logger import Logger
logger = Logger(__name__)
from dotenv import load_dotenv
load_dotenv()

from src.llms.openai import LLMOpenAI


llm = LLMOpenAI()
redis_client = RedisClient()
redis_client.debug_redis()

2025-07-14 10:25:58,476 - src.utils.redis_client - INFO - ✅ ✅ Connected to Redis: True
2025-07-14 10:25:58,482 - src.utils.redis_client - INFO - ✅ ✅ Redis connection: True
2025-07-14 10:25:58,490 - src.utils.redis_client - INFO - ✅ 📋 Redis keys: ['memory:IUf4eLFXW8q72t8pAAAB', 'memory:promptfoo-test-session-823207f7-c471-4fc9-a6aa-2277ea5a7f52', 'memory:YuIgxUbJicq1X7gAAAAD', 'memory:promptfoo-test-session-342883a5-2adb-4c3a-8ee1-bf8ae045f8f9', 'memory:rI_eB5yruJkj5fy_AAAF', 'memory:promptfoo-test-session-cfcc57df-d8e8-4b6a-8d25-5abc5bb0704d', 'memory:Z1hqAIsDkd4gs0hAAABw', 'memory:O2mE2_QSPfwnYC0uAAAD', 'memory:6IlTwKyMwI-V0bDIAAAB', 'memory:IOLGVvZzEaiqpcrJAAAD', 'memory:vnydPWFTT_lZkzpCAAAD', 'memory:promptfoo-test-session-515366c4-09be-4ee7-84a6-c3af1cb23a15', 'memory:promptfoo-test-session-da830615-84c4-4dfd-9ec5-daf81ababbcb', 'memory:c_ShT4NdzlOSkf5yAAAB', 'memory:UKUw4Lcq2CKL0jJjAAAF', 'memory:j5MEsqcf7kXIso6nAAAL', 'memory:promptfoo-test-session-0fc6a8a7-171d-4f39-ae8a-196f89f

True

In [16]:
list_keys_memory = redis_client.redis_client.keys("*")
print(list_keys_memory[:5])

['memory:IUf4eLFXW8q72t8pAAAB', 'memory:promptfoo-test-session-823207f7-c471-4fc9-a6aa-2277ea5a7f52', 'memory:YuIgxUbJicq1X7gAAAAD', 'memory:promptfoo-test-session-342883a5-2adb-4c3a-8ee1-bf8ae045f8f9', 'memory:rI_eB5yruJkj5fy_AAAF']


In [17]:
conversation = redis_client.get('memory:qv5V9tZ80N1fApTkAAAZ')   
conversation

'[{"type": "human", "content": "anh mu\\u1ed1n \\u0111\\u1eb7t l\\u1ecbch d\\u1ecdn nh\\u00e0 s\\u00e1ng th\\u1ee9 3 tu\\u1ea7n sau l\\u00fac 11 gi\\u1edd t\\u1ea1i S\\u1ed1 1 V\\u00f5 V\\u0103n Ng\\u00e2n Th\\u1ee7 \\u0110\\u1ee9c trong 2 ti\\u1ebfng kh\\u1ecfi c\\u1ea7n t\\u00ednh gi\\u00e1 ", "timestamp": "2025-07-03T10:39:43.420762"}, {"type": "ai", "content": "T\\u00f4i c\\u00f3 th\\u1ec3 gi\\u00fap anh \\u0111\\u1eb7t l\\u1ecbch d\\u1ecdn nh\\u00e0. \\u0110\\u1ec3 ho\\u00e0n t\\u1ea5t vi\\u1ec7c n\\u00e0y, t\\u00f4i c\\u1ea7n bi\\u1ebft th\\u00eam m\\u1ed9t v\\u00e0i th\\u00f4ng tin. Anh c\\u00f3 th\\u1ec3 cho t\\u00f4i bi\\u1ebft th\\u1ee9 Ba tu\\u1ea7n sau l\\u00e0 ng\\u00e0y m\\u1ea5y kh\\u00f4ng \\u1ea1?", "timestamp": "2025-07-03T10:39:43.420779"}, {"type": "human", "content": "t\\u1ef1 t\\u00ednh \\u0111i ", "timestamp": "2025-07-03T10:39:43.420786"}, {"type": "ai", "content": "Th\\u1ee9 Ba tu\\u1ea7n sau l\\u00e0 ng\\u00e0y 8 th\\u00e1ng 7 n\\u0103m 2025. Anh mu\\u1ed1n \\

In [18]:
import json 
with open("../src/MCP/schema_summary.json", "r") as f:
    schema = json.load(f)


In [19]:
def get_prompt_conversation(conversation, schema):
    """Create a conversation prompt template with memory placeholder."""
    with open("../src/prompts/prompt_summary.md", "r", encoding="utf-8") as f:
        system_prompt = f.read()
        
    prompt = f"""
        {system_prompt}
        The conversation:
        {conversation}
        Follow output by output_schema JSON schema:
        {schema}

"""
    return prompt


In [20]:
prompt = get_prompt_conversation(conversation, schema['output_schema'])
response = llm.invoke(prompt)
print(response.content)



2025-07-14 10:26:10,248 - src.llms.openai - INFO - ✅ LLM invoked successfully with model: gpt-4o-mini


{
    "session_id": "1",
    "summary": {
        "address": [
            "Số 1 Võ Văn Ngân, Thủ Đức"
        ],
        "date": "2025-07-03T10:39:43Z",
        "service_type": "Đặt lịch đến nhà",
        "summary_detail": "Người dùng muốn đặt lịch đến nhà vào lúc 11 giờ sáng ngày 8 tháng 7 năm 2025 tại địa chỉ Số 1 Võ Văn Ngân, Thủ Đức trong 2 tiếng. Không có yêu cầu dịch vụ nào khác."
    }
}


In [21]:
rep = response.model_dump()['content']
rep

'{\n    "session_id": "1",\n    "summary": {\n        "address": [\n            "Số 1 Võ Văn Ngân, Thủ Đức"\n        ],\n        "date": "2025-07-03T10:39:43Z",\n        "service_type": "Đặt lịch đến nhà",\n        "summary_detail": "Người dùng muốn đặt lịch đến nhà vào lúc 11 giờ sáng ngày 8 tháng 7 năm 2025 tại địa chỉ Số 1 Võ Văn Ngân, Thủ Đức trong 2 tiếng. Không có yêu cầu dịch vụ nào khác."\n    }\n}'

In [22]:
data = json.loads(rep.strip().removeprefix("```json\n").removesuffix("```"))
data

{'session_id': '1',
 'summary': {'address': ['Số 1 Võ Văn Ngân, Thủ Đức'],
  'date': '2025-07-03T10:39:43Z',
  'service_type': 'Đặt lịch đến nhà',
  'summary_detail': 'Người dùng muốn đặt lịch đến nhà vào lúc 11 giờ sáng ngày 8 tháng 7 năm 2025 tại địa chỉ Số 1 Võ Văn Ngân, Thủ Đức trong 2 tiếng. Không có yêu cầu dịch vụ nào khác.'}}

In [47]:
import redis
import os
import sys
sys.path.append(os.path.abspath('..'))
from src.utils.redis_client import RedisClient
from src.utils.logger import Logger
logger = Logger(__name__)
from dotenv import load_dotenv
load_dotenv()

from src.llms.openai import LLMOpenAI


llm = LLMOpenAI()
redis_client = RedisClient()
redis_client.debug_redis()

from pydantic import BaseModel
from langchain_core.memory import BaseMemory
from pydantic import Field,PrivateAttr
from typing import List, Dict, Any
import json 
class SummaryConversation(BaseModel):
    llm: Any = Field(...)
    session_ids: list[str] = Field(...)
    max_token_limit: int = Field(default=150, description="Maximum number of tokens for the summary")
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def get_conversation(self,**kwargs):
        conversation = [redis_client.get(f'{session_id}') for session_id in self.session_ids]
        return conversation
    
    def get_output_schema(self, schema_path: str):
        import json
        with open(schema_path, "r") as f:
            schema = json.load(f)
        return schema['output_schema']
    
    def create_prompt(self, schema):
        conversation = [self.get_conversation()]
        """Create a conversation prompt template with memory placeholder."""
        with open("../src/prompts/prompt_summary.md", "r", encoding="utf-8") as f:
            system_prompt = f.read()
        
        prompt = f"""
            {system_prompt}
            The conversation:
            {self.session_ids}
            {conversation}
            Follow output by output_schema JSON schema:
            {schema}
    """
        return prompt
    
    def summary_llm(self, **kwargs):
        schema = self.get_output_schema("../src/MCP/schema_summary.json")
        prompt = self.create_prompt(schema)
        response = self.llm.invoke(prompt).model_dump()['content']
        response = response.strip().removeprefix("```json\n").removesuffix("```")
        return json.loads(response)


2025-07-14 10:29:27,564 - src.utils.redis_client - INFO - ✅ ✅ Connected to Redis: True
2025-07-14 10:29:27,569 - src.utils.redis_client - INFO - ✅ ✅ Redis connection: True
2025-07-14 10:29:27,581 - src.utils.redis_client - INFO - ✅ 📋 Redis keys: ['memory:IUf4eLFXW8q72t8pAAAB', 'memory:promptfoo-test-session-823207f7-c471-4fc9-a6aa-2277ea5a7f52', 'memory:YuIgxUbJicq1X7gAAAAD', 'memory:promptfoo-test-session-342883a5-2adb-4c3a-8ee1-bf8ae045f8f9', 'memory:rI_eB5yruJkj5fy_AAAF', 'memory:promptfoo-test-session-cfcc57df-d8e8-4b6a-8d25-5abc5bb0704d', 'memory:Z1hqAIsDkd4gs0hAAABw', 'memory:O2mE2_QSPfwnYC0uAAAD', 'memory:6IlTwKyMwI-V0bDIAAAB', 'memory:IOLGVvZzEaiqpcrJAAAD', 'memory:vnydPWFTT_lZkzpCAAAD', 'memory:promptfoo-test-session-515366c4-09be-4ee7-84a6-c3af1cb23a15', 'memory:promptfoo-test-session-da830615-84c4-4dfd-9ec5-daf81ababbcb', 'memory:c_ShT4NdzlOSkf5yAAAB', 'memory:UKUw4Lcq2CKL0jJjAAAF', 'memory:j5MEsqcf7kXIso6nAAAL', 'memory:promptfoo-test-session-0fc6a8a7-171d-4f39-ae8a-196f89f

In [49]:
sessions = list_keys_memory[:5]
SummaryConversation(
    llm=llm, 
    session_ids=sessions).summary_llm()

2025-07-14 10:29:39,963 - src.llms.openai - INFO - ✅ LLM invoked successfully with model: gpt-4o-mini


{'session_id': 'memory:promptfoo-test-session-342883a5-2adb-4c3a-8ee1-bf8ae045f8f9',
 'summary': {'address': ['12 Trần Não, Quận 2, Hồ Chí Minh',
   '1 Hoàng Hoa Thám, Quận Tân Bình, Thành phố Hồ Chí Minh'],
  'date': '2025-06-27T08:55:06Z',
  'service_type': 'Dịch vụ dọn dẹp nhà',
  'summary_detail': 'Người dùng đã yêu cầu đặt dịch vụ dọn dẹp nhà vào lúc 10 giờ sáng ngày mai tại địa chỉ 12 Trần Não, Quận 2, Hồ Chí Minh. Sau đó, có một yêu cầu khác cho dịch vụ dọn dẹp tại địa chỉ 1 Hoàng Hoa Thám, Quận Tân Bình, nhưng không thể hoàn tất ngay lập tức do sự cố kỹ thuật.'}}